In [5]:
import os
import tarfile
import urllib.request


In [6]:
DATA_DIR = os.path.join(os.getcwd(), 'data')
MODELS_DIR = os.path.join(DATA_DIR,  'models')

for dir in [DATA_DIR, MODELS_DIR]:
    if not os.path.exists(dir):
        os.mkdir(dir)

In [9]:
def print_progress(n_blks_tr, blk_size, file_size):
    print(f"\rTotal File Size: {file_size}      Data Read: {n_blks_tr*blk_size}      Percent Complete: {100*n_blks_tr*blk_size/file_size:2.2f}%", end='')

In [10]:
model_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz"
MODEL_DATE = "20200711"
MODEL_NAME = "ssd_resnet101_v1_fpn_640x640_coco17_tpu-8"
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE  + MODEL_DATE + '/' + MODEL_TAR_FILENAME
PATH_TO_MODEL_TAR = os.path.join(MODELS_DIR, MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))

if not os.path.exists(PATH_TO_CKPT):
    print(f"Downloading Model. This may take a while...", end='')
    urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR, print_progress)
    tar_file = tarfile.open(PATH_TO_MODEL_TAR)
    tar_file.extractall(MODELS_DIR)
    tar_file.close()
    os.remove(PATH_TO_MODEL_TAR)
    print("Done!")

LABEL_FILENAME = 'mscoco_label_map.pbtxt'
LABELS_DOWNLOAD_BASE = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
PATH_TO_LABELS = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, LABEL_FILENAME))
if not os.path.exists(PATH_TO_LABELS):
    print("Downloading label file...", end='')
    urllib.request.urlretrieve(LABELS_DOWNLOAD_BASE+LABEL_FILENAME, PATH_TO_LABELS, print_progress)
    print("Done!")

Total File Size: 386527459      Data Read: 386531328      Percent Complete: 100.00%Done!
Total File Size: 5056      Data Read: 8192      Percent Complete: 162.03%Done!


In [16]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from object_detection.utils import label_map_util, config_util, visualization_utils 
from object_detection.builders import model_builder

tf.get_logger().setLevel('ERROR')

In [17]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

In [18]:
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

In [19]:
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

In [20]:
@tf.function
def detect_fn(image):
    """Detect object in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

In [21]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [31]:
import cv2
import matplotlib.pyplot as plt

In [34]:
cap = cv2.VideoCapture(1)
ret, image_np = cap.read()
print(image_np)
plt.imshow(image_np)


None


TypeError: Image data of dtype object cannot be converted to float

In [24]:
import numpy as np

In [29]:
while True:
    # Read image from camera
    ret, image_np = cap.read()

    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()
    # 
    # Convert image to greyscale
    # image_np = np.tile(
    #      np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    visualization_utils.visualize_boxes_and_labels_on_image_array(image_np_with_detections,detections['detection_boxes'][0].numpy(),(detections['detection_classes'][0].numpy() + label_id_offset).astype(int),detections['detection_scores'][0].numpy(),category_index,use_normalized_coordinates=True,max_boxes_to_draw=200,min_score_thresh=.30,agnostic_mode=False)

    cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

StagingError: in user code:

    <ipython-input-20-d71d066036a1>:5 detect_fn  *
        image, shapes = detection_model.preprocess(image)
    /usr/local/lib/python3.8/dist-packages/object_detection/meta_architectures/ssd_meta_arch.py:482 preprocess  *
        normalized_inputs = self._feature_extractor.preprocess(inputs)
    /usr/local/lib/python3.8/dist-packages/object_detection/models/ssd_resnet_v1_fpn_keras_feature_extractor.py:204 preprocess  *
        if resized_inputs.shape.as_list()[3] == 3:

    IndexError: list index out of range
